In [1]:
data = [
    "Pelayanan di kantor ini sangat cepat dan ramah,positif",
    "Aplikasi ini sangat membantu pekerjaan saya,positif",
    "Saya puas dengan kualitas produk yang diberikan,positif",
    "Pengiriman barang sangat tepat waktu,positif",
    "Fitur terbaru aplikasi ini sangat berguna,positif"
]

In [2]:
negative_data = [
    "Pelayanan di kantor ini sangat lambat dan mengecewakan,negatif",
    "Aplikasi sering error dan sulit digunakan,negatif",
    "Saya tidak puas dengan kualitas produk ini,negatif",
    "Pengiriman barang sangat terlambat,negatif",
    "Fitur aplikasi ini tidak berfungsi dengan baik,negatif"
]

In [3]:
neutral_data = [
    "Aplikasi ini memiliki beberapa fitur baru,netral",
    "Saya menggunakan layanan ini setiap hari,netral",
    "Produk ini tersedia dalam beberapa warna,netral",
    "Pengiriman dilakukan melalui jasa ekspedisi,netral",
    "Aplikasi ini diperbarui minggu lalu,netral"
]

In [4]:
unlabeled_data = [
    "Pelayanan yang diberikan cukup memuaskan",
    "Aplikasi ini sering mengalami gangguan",
    "Produk tersebut baru saja diluncurkan",
    "Saya kecewa dengan hasil yang diterima",
    "Pengiriman dilakukan lebih cepat dari perkiraan",
    "Fitur baru masih perlu penyesuaian",
    "Layanan pelanggan mudah dihubungi",
    "Kualitas barang tidak sesuai harapan",
    "Aplikasi ini digunakan oleh banyak orang",
    "Produk tersedia di beberapa toko"
]

In [5]:
!pip install transformers datasets torch scikit-learn pandas


In [6]:
import pandas as pd

# Combine all labeled data
all_labeled_data = data + negative_data + neutral_data

# Prepare data for DataFrame
train_texts = []
train_labels = []
for item in all_labeled_data:
    text, label = item.split(',')
    train_texts.append(text)
    train_labels.append(label)

# Create train DataFrame and save to CSV
train_df_raw = pd.DataFrame({'text': train_texts, 'label': train_labels})
train_df_raw.to_csv('train_sentiment.csv', index=False)

# Create test DataFrame from unlabeled_data and save to CSV
test_df_raw = pd.DataFrame({'text': unlabeled_data})
test_df_raw.to_csv('test_sentiment.csv', index=False)

# Now read the CSV files
train_df = pd.read_csv("train_sentiment.csv")
test_df  = pd.read_csv("test_sentiment.csv")

label_map = {"negatif": 0, "netral": 1, "positif": 2}
train_df["label_id"] = train_df["label"].map(label_map)

In [7]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

train_dataset = Dataset.from_pandas(train_df[["text", "label_id"]])
# Rename 'label_id' to 'label' (singular) to match default data collator expectation
train_dataset = train_dataset.rename_column("label_id", "label")
train_dataset = train_dataset.map(tokenize, batched=True)
# Ensure the format is set for PyTorch, including the 'label' column
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1",
    num_labels=3
)


In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./sentiment_model",
    do_eval=False, # Replaced evaluation_strategy="no" with do_eval=False
    per_device_train_batch_size=8,
    num_train_epochs=5,
    learning_rate=2e-5,
    logging_steps=10,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.917300


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=10, training_loss=0.9173017501831054, metrics={'train_runtime': 149.1172, 'train_samples_per_second': 0.503, 'train_steps_per_second': 0.067, 'total_flos': 385420045500.0, 'train_loss': 0.9173017501831054, 'epoch': 5.0})

In [11]:
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize, batched=True)
test_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

predictions = trainer.predict(test_dataset)
pred_labels = predictions.predictions.argmax(axis=1)

reverse_label_map = {0: "negatif", 1: "netral", 2: "positif"}
test_df["predicted_label"] = [reverse_label_map[i] for i in pred_labels]

test_df.head()


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


,text,predicted_label
0,Pelayanan yang diberikan cukup memuaskan,positif
1,Aplikasi ini sering mengalami gangguan,negatif
2,Produk tersebut baru saja diluncurkan,netral
3,Saya kecewa dengan hasil yang diterima,positif
4,Pengiriman dilakukan lebih cepat dari perkiraan,positif


In [12]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=test_df)

https://docs.google.com/spreadsheets/d/12xnwSPN59U_QMraGfBEY9FI9ZIEyNrq5yoHzbIdgv4s/edit#gid=0


In [17]:
test_df.to_csv("hasil_prediksi_sentimen.csv", index=False)
